In [1]:
.libPaths(R.home('/opt/homebrew/Caskroom/miniforge/base/lib/R/library'))
Output = '/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/PHE Database/2. Coding/2.4. Data Wrangling/Output'
cur_date = '080825'

library(readxl)
library(tidyverse)
library(reshape2)
library(lubridate)
library(oce)

# reading in files
ohio_tmax_2020_df = read_csv("Input/Ohio_Tmax_2020.csv")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Loading required package: gsw

Rows: 1048575 Columns: 4
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (4): DOY, x, y, Value

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [2]:
head(ohio_tmax_2020_df)

DOY,x,y,Value
<dbl>,<dbl>,<dbl>,<dbl>
50,2032326,4815818,76.1
50,2033252,4815818,76.1
50,2030473,4814892,75.9
50,2031399,4814892,76.9
50,2032326,4814892,76.5
50,2033252,4814892,76.5


In [3]:
convert_date = function(df, collection_year){
    # converting the day of the year into a date with the month, day, and year
    
    new_df = df %>%
        mutate(Month = month(as_date(DOY)), Day = mday(as_date(DOY)), Year = collection_year,
           Date = format(make_date(month = Month, day = Day, year = Year), format = "%m/%d/%Y")) %>%
        select(-c('DOY', "Month", "Day", "Year"))

    return(new_df)
    }

# calling fn
converted_tmax_2020_df = convert_date(ohio_tmax_2020_df, 2020)

head(converted_tmax_2020_df)

x,y,Value,Date
<dbl>,<dbl>,<dbl>,<chr>
2032326,4815818,76.1,02/20/2020
2033252,4815818,76.1,02/20/2020
2030473,4814892,75.9,02/20/2020
2031399,4814892,76.9,02/20/2020
2032326,4814892,76.5,02/20/2020
2033252,4814892,76.5,02/20/2020


In [21]:
convert_coordinates = function(df){
    # converting utm coordinates to latitude and longitude

    # zone 14 is for Ohio
    # coordinates_df = data.frame()
    # for (i in 1:length(df$x)){
    #     conversion_object = utm2lonlat(df$x[i], df$y[i], zone = 14)

    #     # saving values
    #     coordinates_vector = c(conversion_object$latitude[[1]], 
    #                                 conversion_object$longitude[[1]])
    #     coordinates_df = rbind(coordinates_df, coordinates_vector)
    # }

    # colnames(coordinates_df) = c("Latitude", "Longitude")

    coordinates_df = df %>%
        mutate(coords = utm2lonlat(easting = x, northing = y, zone = 14),
                longitude = coords$longitude,
                latitude = coords$latitude) %>%
        select(-coords)
    
    return(coordinates_df)
}

# calling fn
final_tmax_2020_df = convert_coordinates(converted_tmax_2020_df)

head(final_tmax_2020_df)

ERROR: [1m[33mError[39m in `mutate()`:[22m
[1m[22m[36mℹ[39m In argument: `coords = utm2lonlat(easting = x, northing = y, zone =
  14)`.
[1mCaused by error:[22m
[1m[22m[33m![39m `coords` must be size 1048575 or 1, not 2.


In [ ]:
# combining dfs
final_tmax_2020_df = cbind(converted_tmax_2020_df[,3:4], tmax_2020_lat_long_df)

head(final_tmax_2020_df)

In [19]:
dim(converted_tmax_2020_df)
dim(tmax_2020_lat_long_df)

[1] 1048575       4

[1] 6 2